# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [35]:
file = "..\weatherpy\weather_output.csv"
weather_df = pd.read_csv(file)
#weather_df.dropna()
weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,santa rosa,0,AR,1569160386,27,-36.62,-64.29,64.77,14.67
1,1,faanui,0,PF,1569160387,81,-16.48,-151.75,80.25,19.15
2,2,northam,67,AU,1569160387,76,-31.65,116.67,53.60,8.05
3,3,kapaa,75,US,1569160713,83,22.08,-159.32,78.80,3.36
4,4,ponta do sol,12,BR,1569160387,36,-20.63,-46.00,89.61,4.72


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [36]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)


In [37]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=70)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
# Filter vacation with max temp above 70 degrees F and less than 80
temp_range = weather_df[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80)]
temp_range.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
3,3,kapaa,75,US,1569160713,83,22.08,-159.32,78.80,3.36
5,5,bambous virieux,75,MU,1569160387,83,-20.34,57.76,71.60,13.87
8,8,banyo,7,CM,1569160387,73,6.75,11.81,74.49,4.88
12,12,vaini,40,IN,1569160714,88,15.34,74.49,78.80,9.17
18,18,kudahuvadhoo,100,MV,1569160715,90,2.67,72.89,78.63,18.70


In [39]:
# Filter by wind speed
weather_nospeed = temp_range[temp_range["Wind Speed"] < 10]
weather_nospeed.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
3,3,kapaa,75,US,1569160713,83,22.08,-159.32,78.80,3.36
8,8,banyo,7,CM,1569160387,73,6.75,11.81,74.49,4.88
12,12,vaini,40,IN,1569160714,88,15.34,74.49,78.80,9.17
25,25,dingle,100,PH,1569160716,93,11.00,122.67,75.39,3.02
26,26,hilo,1,US,1569160716,83,19.71,-155.08,75.20,2.17


In [41]:
# Filter by cloudiness
weather_nocloud = weather_nospeed[weather_nospeed["Cloudiness"] == 0]
weather_nocloud

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
40,40,breznita-motru,0,RO,1569160717,20,44.57,23.22,77.00,3.36
158,158,viransehir,0,TR,1569161026,13,37.24,39.76,73.59,9.04
163,163,huilong,0,CN,1569161027,70,31.15,106.50,70.17,1.99
206,206,guaraniacu,0,BR,1569161127,41,-25.10,-52.88,76.83,5.57
211,211,mirabad,0,IR,1569161128,10,28.27,61.33,72.69,9.40
255,255,poum,0,MK,1569161227,43,41.28,20.71,71.60,2.24
312,312,guerrero negro,0,MX,1569161330,81,27.97,-114.04,70.35,3.40
405,405,shunyi,0,CN,1569161615,82,40.13,116.65,77.00,2.24
506,506,turek,0,PL,1569161826,36,52.02,18.50,75.00,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = weather_nocloud.reset_index(drop=True)
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,40,breznita-motru,0,RO,1569160717,20,44.57,23.22,77.00,3.36
1,158,viransehir,0,TR,1569161026,13,37.24,39.76,73.59,9.04
2,163,huilong,0,CN,1569161027,70,31.15,106.50,70.17,1.99
3,206,guaraniacu,0,BR,1569161127,41,-25.10,-52.88,76.83,5.57
4,211,mirabad,0,IR,1569161128,10,28.27,61.33,72.69,9.40
5,255,poum,0,MK,1569161227,43,41.28,20.71,71.60,2.24
6,312,guerrero negro,0,MX,1569161330,81,27.97,-114.04,70.35,3.40
7,405,shunyi,0,CN,1569161615,82,40.13,116.65,77.00,2.24
8,506,turek,0,PL,1569161826,36,52.02,18.50,75.00,1.01


In [28]:
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,40,breznita-motru,0,RO,1569160717,20,44.57,23.22,77.00,3.36,
1,158,viransehir,0,TR,1569161026,13,37.24,39.76,73.59,9.04,
2,163,huilong,0,CN,1569161027,70,31.15,106.50,70.17,1.99,
3,206,guaraniacu,0,BR,1569161127,41,-25.10,-52.88,76.83,5.57,
4,211,mirabad,0,IR,1569161128,10,28.27,61.33,72.69,9.40,
5,255,poum,0,MK,1569161227,43,41.28,20.71,71.60,2.24,
6,312,guerrero negro,0,MX,1569161330,81,27.97,-114.04,70.35,3.40,
7,405,shunyi,0,CN,1569161615,82,40.13,116.65,77.00,2.24,
8,506,turek,0,PL,1569161826,36,52.02,18.50,75.00,1.01,


In [29]:
hotel_df = hotel_df.loc[:, ~hotel_df.columns.str.contains('^Unnamed')]
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,breznita-motru,0,RO,1569160717,20,44.57,23.22,77.00,3.36,
1,viransehir,0,TR,1569161026,13,37.24,39.76,73.59,9.04,
2,huilong,0,CN,1569161027,70,31.15,106.50,70.17,1.99,
3,guaraniacu,0,BR,1569161127,41,-25.10,-52.88,76.83,5.57,
4,mirabad,0,IR,1569161128,10,28.27,61.33,72.69,9.40,
5,poum,0,MK,1569161227,43,41.28,20.71,71.60,2.24,
6,guerrero negro,0,MX,1569161330,81,27.97,-114.04,70.35,3.40,
7,shunyi,0,CN,1569161615,82,40.13,116.65,77.00,2.24,
8,turek,0,PL,1569161826,36,52.02,18.50,75.00,1.01,


In [30]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{
    "html_attributions": [],
    "next_page_token": "CpQCDwEAAI8IWAfwQRT9mT5YAHTOtRecyffe94g-S80CEk1kJctHyTfoRFWtGjNVunfOQZj7J7L-5NXcSZeefBSMxdlTKfjstOhc4cNeb5FtHsOcDEPU-Btadiz0BN8nS5fCZXEtwPE1w1xyZwoYX8oD81eERuZxw-AF4u9ivN81pQK-_vNIph7it-qKXfAE0Uf7uYmpSyYmJ3KB4X9yzL78X2R_ERm9glzjxjhwHPiJpY4sdbp0XizK7HBnps_lNk_3inH5S9p2t0EU5eRn9BuG6_r8gMQDCwXyrSooELe7NxphV4J4gefjMuhK7kEr0cNNdqqya1DSurDjFiOoLbiiXd6dXj-jLkGqvrhDx4DXztpVHLx2EhByqnawaXm5W5mGU_v-1ZN_GhRrbTNOPxExxQMdwtKV1AYz6sNybw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 44.323909,
                    "lng": 23.804519
                },
                "viewport": {
                    "northeast": {
                        "lat": 44.32522667989273,
                        "lng": 23.80592177989272
                    },
                    "southwest": {
                        "lat": 44.32252702010729,
                        "lng": 23.80322212010728
                  

C:\Users\Saiteja\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 37.2279478,
                    "lng": 39.773182
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.22936427989272,
                        "lng": 39.77452162989272
                    },
                    "southwest": {
                        "lat": 37.22666462010728,
                        "lng": 39.77182197010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "da9714d345ddf8c145775f9785bf6fb44cb3d363",
            "name": "Yukselhan Hotel",
            "photos": [
                {
                    "height": 4032,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/114199451569171718865/photos\">A 

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -25.4025484,
                    "lng": -52.4084859
                },
                "viewport": {
                    "northeast": {
                        "lat": -25.40124422010728,
                        "lng": -52.40717457010727
                    },
                    "southwest": {
                        "lat": -25.40394387989273,
                        "lng": -52.40987422989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "4d27e3e61fd6bd924c9059ca3047e389e86eab04",
            "name": "Lipski Hotel",
            "photos": [
                {
                    "height": 3006,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/117219096778541624034/photos\

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAOL1FitGojtYV6t5dLtyhbYKTCkUSnPkmwLPj_ZFls8Z03qEH43P5Jimmi9IaT86xjJxM_0p-kNcsN_HPUv0fyS0DX93cQvyrUqkYZuJDIsYmOCJR5kavbi2FOP3DH5ISmHcL8RHSsq2YIy2rG-EvISr1QCjIJ5e1fgMMJaPHr_vJNZXQy0gXaD3gIEsoV8dM-NVJyX5YfxRC6TsuM1paOOeWM3gbaTfb2rfXfEzsIu7EzMEls3eSdkpiAEbFF7lQe8F0Rg0o8atxc7M2gUAMakG7qlSe4HT1j94oI_M1Qx_-LaLsRSlY_oX0L05a_uJHdCF6IMRQCPvS18PuVE-QIDMRjEIxS5rUpHBW_3OnF6XWIf9JUyQQ2zEUMPjggnQewOF47NKQQHPq_jYh9wbk_0SEEmeUX1PZHJ9FcyAPWoa3t8aFBRLFUWTyaM3vSgENt2JQ6qpEG93",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 41.0431589,
                    "lng": 21.2550375
                },
                "viewport": {
                    "northeast": {
                        "lat": 41.04457242989272,
                        "lng": 21.25657297989272
                

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 28.001048,
                    "lng": -114.0138683
                },
                "viewport": {
                    "northeast": {
                        "lat": 28.00238187989272,
                        "lng": -114.0121708701073
                    },
                    "southwest": {
                        "lat": 27.99968222010728,
                        "lng": -114.0148705298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "0d92642a7e1195a06b0932a8fa1ebac253cf5178",
            "name": "The Halfway Inn",
            "photos": [
                {
                    "height": 2448,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/117756932328837627923/photos\

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAADwqujmR3RkWm-UXaotpRXyBQmABOj6cmHii0ZiqP8dOdMqsquzeN8Fh8uasaExNFqqDZlsf9Wg65TlpJWTqVsB7G66vQkhoRR1uYwBPo1ObjPWeIrc9ZK-9hJQVuCrlpVkSPnvwsxJiMg1IQ7ziSw6WTEj6AofAx8ugfnhcopqhUxYOeIFQbr272VLgcAUfZPUSSa0i6l7iDniO8asstvhAsnWVbMSbyf-k41eD1r6Ieu6QHaD8L3vctdQjRymUiRzsbJEEyu8iCGkVVVQLTHXB56SZqu0pkHvWtpMmO8fOrkoI2N-IFWrBtrI9KrfKwrj2Lypixqn3kXILp7M8szl90q3UVOCoDKjda2E41FSg51oxQ1Uk6zt_tH4tltus6s9n-7ugTn-FIqhJT1kmTmUSEHz2ubMOc3HvprFMeG5vaBkaFLXMxJhFAD4F7Yj1LKiLLmvnNVvk",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 39.934951,
                    "lng": 116.463608
                },
                "viewport": {
                    "northeast": {
                        "lat": 39.93630347989272,
                        "lng": 116.4648169298927
                    },
                    "southwest": {
                        "lat": 39.93360382010728,
                        

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAANGHJSvkJaE28DQoVVCN9v9ppALxYQj4uLl0FcCKVkwxp_KiKd0M3MBNaKgnXCZaMYr5DpSreHW-M7_k47lC45WCgIRfQOHylOCrS17V6M0GdJNqzGFLXqEwyfL8zecZ1TCW8nbB3E7LVxT1q_Sl5F5gk1LkOOMhwQO3Z6KItER2pGmTrS5YoOY0QmubIkSc61jr_KfKjk8uQsWuqepBGZnsNPkhAuxibhAqF0cMJ6c36OS96v2QmpefcJPOqjY8lnT-3p7BRYBx0pnK2DG07Nic9YjhMM0HVUHXxS8vLUtFXYZr_sxmTL8aq3MeNtkd4z5qmlx-P7fkUhMnekGHPAYMvYiGt1rgpvYw-WWSMMCy-OtWgZjfM54Hf4fOGYuWGlqIMjX8ltb-G_03kPT8IxoSEDsOB1dqGDWu17BUrQehS3IaFFaOFzNTWSgGK_m9atYXTzyy18SE",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 52.33679,
                    "lng": 18.305591
                },
                "viewport": {
                    "northeast": {
                        "lat": 52.33862592989272,
                        "lng": 18.30740007989272
                    },
                    "southwest": {
                        "lat": 52.33592627010727,
                        "l

In [31]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,breznita-motru,0,RO,1569160717,20,44.57,23.22,77.00,3.36,Hotel Rexton
1,viransehir,0,TR,1569161026,13,37.24,39.76,73.59,9.04,Yukselhan Hotel
2,huilong,0,CN,1569161027,70,31.15,106.50,70.17,1.99,Hongyuan Inn
3,guaraniacu,0,BR,1569161127,41,-25.10,-52.88,76.83,5.57,Lipski Hotel
4,mirabad,0,IR,1569161128,10,28.27,61.33,72.69,9.40,
5,poum,0,MK,1569161227,43,41.28,20.71,71.60,2.24,Hotel Sumski Feneri
6,guerrero negro,0,MX,1569161330,81,27.97,-114.04,70.35,3.40,The Halfway Inn
7,shunyi,0,CN,1569161615,82,40.13,116.65,77.00,2.24,Saga Hotel
8,turek,0,PL,1569161826,36,52.02,18.50,75.00,1.01,Hotel Mikorzyn


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))